In [2]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [3]:
!python -V

Python 3.12.3


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [22]:
year = 2023
month = 3
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'./outputs/output_{year:04d}-{month:02d}.parquet'

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [23]:
df = read_data(input_file)
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [25]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [26]:
float(y_pred.std())

6.247488852238703

In [29]:

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predict'] = y_pred

df_result

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [31]:
!ls -lh ./outputs/output_2023-03.parquet

-rw-r--r-- 1 rpeklov rpeklov 66M Feb 19 16:52 ./outputs/output_2023-03.parquet


In [33]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1476 bytes to starter.py
